# Cantilever beam with 4C

In this example, we will be creating

We will first create the mesh

In [35]:
# We have to import the required classes and functions from BeamMe
from beamme.core.boundary_condition import BoundaryCondition
from beamme.core.conf import bme
from beamme.core.mesh import Mesh
from beamme.four_c.element_beam import Beam3rLine2Line2
from beamme.four_c.header_functions import (
    set_header_static,
    set_runtime_output,
)
from beamme.core.function import Function
from beamme.four_c.input_file import InputFile
from beamme.four_c.material import MaterialReissner
from beamme.four_c.run_four_c import clean_simulation_directory
from beamme.mesh_creation_functions.beam_line import create_beam_mesh_line
from pathlib import Path
from utils.lecture_utils import run_four_c

# In the beginning, we create the mesh container which will hold all elements,
# materials, and boundary conditions.
mesh = Mesh()

# The Load will be linearily increased from 0 to 1 over 1 pseudo-time unit.
function_load = Function([{"SYMBOLIC_FUNCTION_OF_TIME": "t"}])
mesh.add(function_load)

# Here we define the material properties for the beam elements.
# We use a Reissner beam material with specified radius, Young's modulus,
material = MaterialReissner(radius=0.01, youngs_modulus=10000.0)

# Next, we create a straight beam connecting two points in space.
# For spatial discretization, we use 3 linear two-noded beam elements.
beam_set = create_beam_mesh_line(
    mesh, Beam3rLine2Line2, material, [0, 0, 0], [1, 0, 0], n_el=100
)

# To fix the cantilver beam, we apply Dirichlet boundary conditions to all
# positions and rotations at one node.
mesh.add(
    BoundaryCondition(
        beam_set["start"],
        {
            "NUMDOF": 6,
            "ONOFF": [1, 1, 1, 1, 1, 1],
            "VAL": [0, 0, 0, 0, 0, 0],
            "FUNCT": [0, 0, 0, 0, 0, 0],
        },
        bc_type=bme.bc.dirichlet,
    )
)

# The concentrated load at the end an be applied as a Neuman boundary
# condition in z-direction.
mesh.add(
    BoundaryCondition(
        beam_set["end"],
        {
            "NUMDOF": 6,
            "ONOFF": [0, 0, 1, 0, 0, 0],
            "VAL": [0, 0, 0.0001, 0, 0, 0],
            "FUNCT": [0, 0, function_load, 0, 0, 0],
        },
        bc_type=bme.bc.neumann,
    )
)

# Next, we create the input file and set simulation parameters such as
# number of time steps, solver tolerances and output options.
input_file = InputFile()
input_file.add(mesh)
set_header_static(input_file, total_time=1.0, n_steps=4, max_iter=20,tol_residuum=1e-10, tol_increment=1,create_nox_file=False, predictor="ConstDis")
set_runtime_output(input_file)

# Now we are ready to dump the input file to disk
simulation_directory = Path("./simulation_directory/")
input_file_path = simulation_directory / "cantilever.4C.yaml"
clean_simulation_directory(simulation_directory)
input_file.dump(input_file_path)

# Finally, we can run the 4C simulation using the created input file.
# return_code = run_four_c(
#     input_file_path.absolute(),
#     simulation_directory,
#     four_c_exe="/data/a11bivst/dev/4C/release/4C",
#     output_name="xxx",
# )

import subprocess
import os
four_c_exe="/data/a11bivst/dev/4C/release/4C"
   

    # Setup paths and actual command to run
os.makedirs(simulation_directory, exist_ok=True)
command = [
        four_c_exe,
        input_file_path.absolute(),
        "xxx",
    ]

with open(simulation_directory / "xxx.log", "w") as logfile:
    print("Start simulation")
    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,  # merge stderr into stdout (optional)
        text=True,                 # get str instead of bytes
        bufsize=1,                 # line-buffered
        cwd=simulation_directory
    )

    nonlinear_solver_step_count = 0
    is_error = False
    finished = False
    for line in process.stdout:
        line = line.rstrip("\n")

        # 2) Write to log file
        logfile.write(line + "\n")

        # 3) Flush file so log is always up to date
        logfile.flush()

        # 4) Process the line however you want
        if "Nonlinear Solver Step" in line:
            nonlinear_solver_step_count = int(line.split(" ")[4])
        elif "||F||" in line:
            if not nonlinear_solver_step_count == 0:
                residuum = float(line.split(" ")[2])
                print(
                    f"  Nonlinear Solver Step {nonlinear_solver_step_count}: Residuum = {residuum:.3e}"
                )
        elif "Finalised step" in line:
            split = line.split(" ")
            step = int(split[2])
            time = float(split[7])
            print(f"Finished time step {step} for time {time:.3e}")
        elif "OK (0)" in line:
            finished = True
        elif "=========================================================================" in line and not finished:
            if is_error:
                print(line)
            is_error = not is_error
        
        if is_error:
            print(line)

    returncode = process.wait()


ImportError: cannot import name 'run_four_c' from 'utils.lecture_utils' (/home/a11bivst/workspace/input/beamme/examples/utils/lecture_utils.py)

The `Mesh` class is the core BeamMe class that will hold all the nodes, elements, materials, and geometry sets for the created geometries.

In [ ]:
from beamme.core.mesh import Mesh

## Straight lines

We already have everything we need to create basic geometries.
Let's start of with the most basic one, a straight line.
For that we need the `create_beam_mesh_line` function.
Mesh generation functions always require to provide a mesh to add the created geometry to, a beam type and a beam material.
In this example we create a line between the points $\vv{p} = [0,0,0]$ and $\vv{q}=[1,0,0]$ with 3 equally spaced beam elements:

In [ ]:
from beamme.mesh_creation_functions.beam_line import create_beam_mesh_line

mesh = Mesh()
material = beam_mat(radius=0.01, youngs_modulus=1.0, nu=0.3, density=1.0)
create_beam_mesh_line(mesh, beam_type, material, [0, 0, 0], [1, 0, 0], n_el=3)

We can see that the mesh creation function returned a dictionary containing `GeometrySet`s.
These geometry sets can be used to define boundary conditions.
In this case we get the following sets:
- `start`: A geometry set referring to the start node of the line
- `end`: A geometry set referring to the end node of the line
- `line`: A geometry set referring to all created beam elements along the line

Later we will dive closer into `GeometrySets`.

We can directly have a look at the created geometry with the `Mesh.display_pyvista` method:

In [ ]:
mesh.display_pyvista()

TODO: describe vtu output

## TODO